In [1]:
import numpy as np
from collections import Counter, defaultdict
from sklearn.model_selection import train_test_split
import math
import pandas as pd
from sklearn import preprocessing
import random

In [2]:
data= pd.read_csv("/Users/SKU/Desktop/fall2017/data mining/Project 3/project3_dataset2.txt", sep="\t", header=None)
print(data)

       0      1      2      3        4   5      6       7   8  9
0    132   6.20   6.47  36.21  Present  62  30.77   14.14  45  0
1    123   0.05   4.61  13.69   Absent  51  23.23    2.78  16  0
2    128   0.50   3.70  12.81  Present  66  21.25   22.73  28  0
3    114   9.60   2.51  29.18   Absent  49  25.67   40.63  46  0
4    150   0.30   6.38  33.99  Present  62  24.64    0.00  50  0
5    136   8.80   4.69  36.07  Present  38  26.56    2.78  63  1
6    144   0.76  10.53  35.66   Absent  63  34.35    0.00  55  1
7    134  11.79   4.01  26.57  Present  38  21.79   38.88  61  1
8    126   8.75   6.53  34.02   Absent  49  30.25    0.00  41  1
9    164   5.60   3.17  30.98  Present  44  25.99   43.20  53  1
10   178  20.00   9.78  33.55   Absent  37  27.29    2.88  62  1
11   136   3.99   2.58  16.38  Present  53  22.41   27.67  36  0
12   128   0.00   2.63  23.88   Absent  45  21.59    6.54  57  0
13   146   0.64   4.82  28.02   Absent  60  28.11    8.23  39  1
14   112   4.46   7.18  2

In [3]:
# Separate Attribute and Label
labels = data[data.columns[-1]]
data = data.ix[:, :len(data.columns)-2]
print(data.head())

     0     1     2      3        4   5      6      7   8
0  132  6.20  6.47  36.21  Present  62  30.77  14.14  45
1  123  0.05  4.61  13.69   Absent  51  23.23   2.78  16
2  128  0.50  3.70  12.81  Present  66  21.25  22.73  28
3  114  9.60  2.51  29.18   Absent  49  25.67  40.63  46
4  150  0.30  6.38  33.99  Present  62  24.64   0.00  50


/Users/SKU/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
# Handeling Categorical Data

#Select Categorical Columns from the dataframe
obj_df = data.select_dtypes(include=['object']).copy()

#One-Hot Encoding for categorical data
for idx in obj_df.columns:
    obj_df= pd.get_dummies(obj_df,columns=[idx],drop_first=False)

obj_dataset = obj_df.values.tolist()


In [5]:
# Handeling Numerical Data
#Select numerical data from the data
num_df = data.select_dtypes(include=['int64','float64']).copy()

# Normalize Numeric Data
if (len(num_df.columns) !=0):
    num_dataset = preprocessing.normalize(num_df, norm='l2')
    num_dataset
else:
    num_dataset = []


In [6]:
# Dataset Format to be used for algorithm
if len(num_dataset)!= 0:
    dataset = np.column_stack((num_dataset, obj_dataset))
    dataset = np.column_stack((dataset, labels))
else:
    dataset = np.column_stack((obj_dataset, labels))


In [7]:
label_col = len(dataset[0])

In [8]:
lengthOfData = len(dataset)

In [9]:
train, test = train_test_split(dataset, test_size=0.3)

In [10]:
train.shape, test.shape

((323, 11), (139, 11))

In [11]:
output = train[:,(len(dataset[0])-1)]

In [12]:
def separateByClass(train):
    separated = {}
    for i in range(len(train)):
        vector = train[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

In [13]:
def getProbability(x,mean,var):
    denominator = (2 * math.pi * var) ** 0.5
    numerator = np.exp( - (x - mean) ** 2 / (2 * var))
    return numerator / denominator
    

In [14]:
 classSeperated =  separateByClass(train)

In [15]:
# divide entire data into n folds
def cross_validation(n_folds):
    updated_datasets = list(dataset)

    size_fold = math.floor(len(updated_datasets)/n_folds)    #training_dataset)/n_folds)
    new_dataset = []
    for i in range(0, n_folds):
        count = 0
        fold = []
        while count < size_fold:
            r = random.randint(0, len(updated_datasets)-1)
            fold.append(updated_datasets[r])
            updated_datasets.pop(r)
            count +=1
        new_dataset.append(fold)
    return new_dataset

In [16]:
def get_foldset(updatedset, test_index): 
    foldset = []
    
    #remove the test set and combine other lists for train 
    newset = list(updatedset)
    newset.pop(test_index)
    foldset = sum(newset, [])
 
    return foldset

In [18]:
# cross validation part
random.seed(1234)
n_fold = 10
validated_dataset = cross_validation(n_fold)
accuracies = []
precisions = []
recalls = []
f1_measures = []
for i in range(0, n_fold):
    print("beginnning of  fold : "+ str(i+1))
    test = validated_dataset[i]
    train = get_foldset(validated_dataset, i)
    
    # naive bayes algo
    true_positive = 0
    false_positive = 0
    f_n = 0

    pclass0 = float(len(classSeperated[0.0])) / len(train)
    pclass1 = float(len(classSeperated[1.0])) / len(train)

    class1Data = classSeperated[1.0]
    meanClass1 = np.mean(class1Data,axis=0)
    varClass1 = np.var(class1Data,axis=0)

    class0Data = classSeperated[0.0]
    meanClass0 = np.mean(class0Data,axis=0)
    varClass0 = np.var(class0Data,axis=0)

    #print(getProbability(train[0][:-1],meanClass0[:-1], varClass0[:-1]))
    #print(pclass0, pclass1)
    correctPredicted = 0
    
    for i in range(len(test)):
    
        pXClass1 = np.prod(getProbability(test[i][:-1], meanClass1[:-1], varClass1[:-1]))
        pXClass0 = np.prod(getProbability(test[i][:-1], meanClass0[:-1], varClass0[:-1]))
    
        pX = (pclass0 * pXClass0) + (pclass1 * pXClass1)
    
        pclass1X = (pclass1 * pXClass1) / pX
        pclass0X = (pclass0 * pXClass0) / pX
    
        expected = test[i][-1]
    
        if pclass1X > pclass0X:
            result = 1.0
    
        else:
            result = 0.0
    
        if expected == result:
            correctPredicted += 1
        
        if (expected == 1.0 and result == 1.0):
            true_positive +=1
        
        if expected == 0.0 and result == 1.0:
            false_positive +=1
        
        if expected == 1.0 and result == 0.0:
            f_n += 1    
        
    Accuracy = float(correctPredicted) / len(test)
    Precision = float(true_positive)/ (true_positive + false_positive)
    Recall = float(true_positive)/ (true_positive + f_n)
    f1_measure = float((2 * true_positive ))/ ((2 * true_positive ) + f_n + false_positive )
    
    #append result in the list for all the folds
    accuracies.append(Accuracy)
    precisions.append(Precision)
    recalls.append(Recall)
    f1_measures.append(f1_measure)
        

resultant_accuracy = sum(accuracies)/ len(accuracies)
resultant_precision = sum(precisions)/ len(precisions)
resultant_recall = sum(recalls)/ len(recalls)
resultant_f1measure = sum(f1_measures)/len(f1_measures)

print("Mean accuracy::" + str(resultant_accuracy))
print("Mean precision::" + str(resultant_precision))
print("Mean recall::" + str(resultant_recall))
print("Mean f1::" + str(resultant_f1measure))
    

    

beginnning of  fold : 1
beginnning of  fold : 2
beginnning of  fold : 3
beginnning of  fold : 4
beginnning of  fold : 5
beginnning of  fold : 6
beginnning of  fold : 7
beginnning of  fold : 8
beginnning of  fold : 9
beginnning of  fold : 10
Mean accuracy::0.717391304347826
Mean precision::0.5717527374586199
Mean recall::0.6494826088943736
Mean f1::0.6049914359867439
